##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Усовершенствованная автоматическая дифференциация

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/advanced_autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/advanced_autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

Руководство по [автоматической дифференциации](autodiff.ipynb) включает в себя все необходимое для расчета градиентов. Это руководство фокусируется на более глубоких, менее распространенных функциях API `tf.GradientTape` .

## Настроить

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 6)

## Управление градиентной записью

В руководстве по [автоматическому дифференцированию](autodiff.ipynb) вы увидели, как контролировать, какие переменные и тензоры отслеживаются лентой при построении расчета градиента.

У ленты также есть методы, чтобы управлять записью.

Если вы хотите остановить запись градиентов, вы можете использовать `GradientTape.stop_recording()` чтобы временно приостановить запись.

Это может быть полезно для уменьшения накладных расходов, если вы не хотите различать сложные операции в середине вашей модели. Это может включать вычисление метрики или промежуточного результата:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  x_sq = x * x
  with t.stop_recording():
    y_sq = y * y
  z = x_sq + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

Если вы хотите начать все сначала, используйте `reset()` . Простой выход из блока градиентной ленты и перезапуск обычно проще для чтения, но вы можете использовать `reset` если выход из блока ленты затруднен или невозможен.

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)
reset = True

with tf.GradientTape() as t:
  y_sq = y * y
  if reset:
    # Throw out all the tape recorded so far
    t.reset()
  z = x * x + y_sq

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Стоп градиент

В отличие от глобальных элементов управления ленты выше, функция `tf.stop_gradient` намного более точна. Его можно использовать для предотвращения прохождения градиентов по определенному пути без необходимости доступа к самой ленте:

In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as t:
  y_sq = y**2
  z = x**2 + tf.stop_gradient(y_sq)

grad = t.gradient(z, {'x': x, 'y': y})

print('dz/dx:', grad['x'])  # 2*x => 4
print('dz/dy:', grad['y'])

## Пользовательские градиенты

В некоторых случаях вы можете захотеть контролировать, как именно рассчитываются градиенты, а не использовать значения по умолчанию. Эти ситуации включают в себя:

- Там нет определенного градиента для новой операции, которую вы пишете.
- Расчеты по умолчанию численно нестабильны.
- Вы хотите кэшировать дорогостоящие вычисления с прямого прохода.
- Вы хотите изменить значение (например, используя: `tf.clip_by_value` , `tf.math.round` ) без изменения градиента.

Для написания новой операции вы можете использовать `tf.RegisterGradient` чтобы создать свою собственную. Смотрите эту страницу для деталей. (Обратите внимание, что реестр градиентов является глобальным, поэтому изменяйте его с осторожностью.)

Для последних трех случаев вы можете использовать `tf.custom_gradient` .


Вот пример, который применяет `tf.clip_by_norm` к промежуточному градиенту.

In [ ]:
# Establish an identity operation, but clip during the gradient pass
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward

v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


Смотрите декоратор `tf.custom_gradient` для более подробной информации.

## Несколько лент

Несколько лент взаимодействуют без проблем. Например, здесь каждая лента просматривает свой набор тензоров:

In [ ]:
x0 = tf.constant(0.0)
x1 = tf.constant(0.0)

with tf.GradientTape() as tape0, tf.GradientTape() as tape1:
  tape0.watch(x0)
  tape1.watch(x1)

  y0 = tf.math.sin(x0)
  y1 = tf.nn.sigmoid(x1)

  y = y0 + y1

  ys = tf.reduce_sum(y)

In [ ]:
tape0.gradient(ys, x0).numpy()   # cos(x) => 1.0

In [ ]:
tape1.gradient(ys, x1).numpy()   # sigmoid(x1)*(1-sigmoid(x1)) => 0.25

### Градиенты высшего порядка

Операции внутри контекстного менеджера `GradientTape` записываются для автоматической дифференциации. Если градиенты вычисляются в этом контексте, то вычисление градиента также записывается. В результате точно такой же API работает и для градиентов более высокого порядка. Например:

In [ ]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    y = x * x * x

  # Compute the gradient inside the outer `t2` context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t1.gradient(y, x)
d2y_dx2 = t2.gradient(dy_dx, x)

print('dy_dx:', dy_dx.numpy())  # 3 * x**2 => 3.0
print('d2y_dx2:', d2y_dx2.numpy())  # 6 * x => 6.0

Хотя это дает вам вторую производную от *скалярной* функции, этот шаблон не обобщается для получения матрицы Гессе, поскольку `GradientTape.gradient` вычисляет только градиент скаляра. Чтобы построить гессиан, см. [Гессианский пример в](#hessian) разделе о [якобиане](#jacobians) .

«Вложенные вызовы `GradientTape.gradient` » - это хороший шаблон, когда вы вычисляете скаляр из градиента, а затем полученный скаляр действует как источник для второго вычисления градиента, как в следующем примере.


#### Пример: регуляризация входного градиента

Многие модели подвержены «состязательным примерам». Этот набор методов изменяет ввод модели, чтобы запутать вывод модели. Самая [простая реализация](https://www.tensorflow.org/tutorials/generative/adversarial_fgsm) делает один шаг вдоль градиента вывода относительно ввода; «градиент ввода».

Одним из методов повышения устойчивости к состязательным примерам является [регуляризация входного градиента](https://arxiv.org/abs/1905.11468) , которая пытается минимизировать величину входного градиента. Если входной градиент небольшой, то изменение выходного сигнала также должно быть небольшим.

Ниже приведена наивная реализация регуляризации входного градиента. Реализация:

1. Рассчитайте градиент выхода относительно входа, используя внутреннюю ленту.
2. Рассчитайте величину этого входного градиента.
3. Рассчитайте градиент этой величины относительно модели.

In [ ]:
x = tf.random.normal([7, 5])

layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

In [ ]:
with tf.GradientTape() as t2:
  # The inner tape only takes the gradient with respect to the input,
  # not the variables.
  with tf.GradientTape(watch_accessed_variables=False) as t1:
    t1.watch(x)
    y = layer(x)
    out = tf.reduce_sum(layer(x)**2)
  # 1. Calculate the input gradient.
  g1 = t1.gradient(out, x)
  # 2. Calculate the magnitude of the input gradient.
  g1_mag = tf.norm(g1)

# 3. Calculate the gradient of the magnitude with respect to the model.
dg1_mag = t2.gradient(g1_mag, layer.trainable_variables)

In [ ]:
[var.shape for var in dg1_mag]

## якобианы


Все предыдущие примеры брали градиенты скалярной мишени относительно некоторого тензора (ов) источника.

[Матрица Якоби](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) представляет градиенты вектор-функции. Каждая строка содержит градиент одного из элементов вектора.

Метод `GradientTape.jacobian` позволяет эффективно вычислять матрицу Якоби.

Обратите внимание, что:

- Как `gradient` : аргумент `sources` может быть тензором или контейнером тензоров.
- В отличие от `gradient` : `target` тензор должен быть одним тензором.

### Скалярный источник

В качестве первого примера приведу якобиан вектора-мишени относительно скалярного источника.

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)
delta = tf.Variable(0.0)

with tf.GradientTape() as tape:
  y = tf.nn.sigmoid(x+delta)

dy_dx = tape.jacobian(y, delta)

Когда вы берете якобиан относительно скаляра, результат имеет форму **цели** и дает градиент каждого элемента относительно источника:

In [ ]:
print(y.shape)
print(dy_dx.shape)

In [ ]:
plt.plot(x.numpy(), y, label='y')
plt.plot(x.numpy(), dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

### Тензорный источник

Независимо от того, является ли вход скалярным или тензорным, `GradientTape.jacobian` эффективно рассчитывает градиент каждого элемента источника по отношению к каждому элементу объекта (ов).

Например, выход этого слоя имеет форму `(10, 7)` :

In [ ]:
x = tf.random.normal([7, 5])
layer = tf.keras.layers.Dense(10, activation=tf.nn.relu)

with tf.GradientTape(persistent=True) as tape:
  y = layer(x)

y.shape

И форма ядра слоя `(5, 10)` :

In [ ]:
layer.kernel.shape

Форма якобиана вывода по отношению к ядру - эти две формы, соединенные вместе:

In [ ]:
j = tape.jacobian(y, layer.kernel)
j.shape

Если вы суммируете по измерениям цели, у вас останется градиент суммы, который был бы рассчитан `GradientTape.gradient` :

In [ ]:
g = tape.gradient(y, layer.kernel)
print('g.shape:', g.shape)

j_sum = tf.reduce_sum(j, axis=[0, 1])
delta = tf.reduce_max(abs(g - j_sum)).numpy()
assert delta < 1e-3
print('delta:', delta)

<a id="hessian"> </a>

#### Пример: гессиан

While `tf.GradientTape` doesn't give an explicit method for constructing a Hessian matrix it's possible to build one using the `GradientTape.jacobian` method.

Примечание: матрица Гессе содержит `N**2` параметров. По этой и другим причинам это не практично для большинства моделей. Этот пример включен больше как демонстрация того, как использовать метод `GradientTape.jacobian` , и не является подтверждением прямой гессианской оптимизации. Произведение вектора Гессиана может быть [эффективно рассчитано с помощью вложенных лент](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/benchmarks/resnet50/hvp_test.py) и является гораздо более эффективным подходом к оптимизации второго порядка.


In [ ]:
x = tf.random.normal([7, 5])
layer1 = tf.keras.layers.Dense(8, activation=tf.nn.relu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.relu)

with tf.GradientTape() as t2:
  with tf.GradientTape() as t1:
    x = layer1(x)
    x = layer2(x)
    loss = tf.reduce_mean(x**2)

  g = t1.gradient(loss, layer1.kernel)

h = t2.jacobian(g, layer1.kernel)

In [ ]:
print(f'layer.kernel.shape: {layer1.kernel.shape}')
print(f'h.shape: {h.shape}')

Чтобы использовать этот гессиан для шага метода Ньютона, вы сначала должны сгладить его оси в матрице и сгладить градиент в векторе:

In [ ]:
n_params = tf.reduce_prod(layer1.kernel.shape)

g_vec = tf.reshape(g, [n_params, 1])
h_mat = tf.reshape(h, [n_params, n_params])

Матрица Гессе должна быть симметричной:

In [ ]:
def imshow_zero_center(image, **kwargs):
  lim = tf.reduce_max(abs(image))
  plt.imshow(image, vmin=-lim, vmax=lim, cmap='seismic', **kwargs)
  plt.colorbar()

In [ ]:
imshow_zero_center(h_mat)

Шаг обновления метода Ньютона показан ниже.

In [ ]:
eps = 1e-3
eye_eps = tf.eye(h_mat.shape[0])*eps

Примечание: на [самом деле не инвертируйте матрицу](https://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) .

In [ ]:
# X(k+1) = X(k) - (∇²f(X(k)))^-1 @ ∇f(X(k))
# h_mat = ∇²f(X(k))
# g_vec = ∇f(X(k))
update = tf.linalg.solve(h_mat + eye_eps, g_vec)

# Reshape the update and apply it to the variable.
_ = layer1.kernel.assign_sub(tf.reshape(update, layer1.kernel.shape))

Хотя это относительно просто для одной `tf.Variable` , применение ее к нетривиальной модели потребует тщательной конкатенации и нарезки для получения полного гессиана по нескольким переменным.

### Пакетный якобиан

В некоторых случаях вы хотите взять якобиан каждого из стека целей относительно стека источников, где якобианы для каждой пары цели-источника независимы.

Например, здесь вход `x` имеет форму `(batch, ins)` , а выход `y` имеет форму `(batch, outs)` .


In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = layer2(y)

y.shape

Полный якобиан `y` отношению к `x` имеет форму `(batch, ins, batch, outs)` , даже если вы только хотите `(batch, ins, outs)` .

In [ ]:
j = tape.jacobian(y, x)
j.shape

Если градиенты каждого элемента в стеке независимы, то каждый `(batch, batch)` фрагмент этого тензора является диагональной матрицей:

In [ ]:
imshow_zero_center(j[:, 0, :, 0])
_ = plt.title('A (batch, batch) slice')

In [ ]:
def plot_as_patches(j):
  # Reorder axes so the diagonals will each form a contiguous patch.
  j = tf.transpose(j, [1, 0, 3, 2])
  # Pad in between each patch.
  lim = tf.reduce_max(abs(j))
  j = tf.pad(j, [[0, 0], [1, 1], [0, 0], [1, 1]],
             constant_values=-lim)
  # Reshape to form a single image.
  s = j.shape
  j = tf.reshape(j, [s[0]*s[1], s[2]*s[3]])
  imshow_zero_center(j, extent=[-0.5, s[2]-0.5, s[0]-0.5, -0.5])

plot_as_patches(j)
_ = plt.title('All (batch, batch) slices are diagonal')

Чтобы получить желаемый результат, вы можете суммировать по дублированному размеру `batch` или выбрать диагонали с помощью `tf.einsum` .


In [ ]:
j_sum = tf.reduce_sum(j, axis=2)
print(j_sum.shape)
j_select = tf.einsum('bxby->bxy', j)
print(j_select.shape)

Было бы гораздо эффективнее сделать расчет без дополнительного измерения в первую очередь. Метод `GradientTape.batch_jacobian` делает именно это.

In [ ]:
jb = tape.batch_jacobian(y, x)
jb.shape

In [ ]:
error = tf.reduce_max(abs(jb - j_sum))
assert error < 1e-3
print(error.numpy())

Caution: `GradientTape.batch_jacobian` only verifies that the first dimension of the source and target match. It doesn't check that the gradients are actually independent. It's up to the user to ensure they only use `batch_jacobian` where it makes sense. For example adding a `layers.BatchNormalization` destroys the independence, since it normalizes across the `batch` dimension:

In [ ]:
x = tf.random.normal([7, 5])

layer1 = tf.keras.layers.Dense(8, activation=tf.nn.elu)
bn = tf.keras.layers.BatchNormalization()
layer2 = tf.keras.layers.Dense(6, activation=tf.nn.elu)

with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
  tape.watch(x)
  y = layer1(x)
  y = bn(y, training=True)
  y = layer2(y)

j = tape.jacobian(y, x)
print(f'j.shape: {j.shape}')

In [ ]:
plot_as_patches(j)

_ = plt.title('These slices are not diagonal')
_ = plt.xlabel("Don't use `batch_jacobian`")

В этом случае `batch_jacobian` все еще выполняется и возвращает *что-то* с ожидаемой формой, но его содержимое имеет неясный смысл.

In [ ]:
jb = tape.batch_jacobian(y, x)
print(f'jb.shape: {jb.shape}')